# 朴素贝叶斯
优点：数据较少的情况下仍然有效

缺点：对于输入数据的准备方式较为敏感
## 一般过程：
1. 收集数据
2. 准备数据：数值型或布尔型
3. 分析数据
4. 训练算法：计算不同的独立特征的条件概率
5. 计算错误率

## 原理
计算不同的数据独立特征的条件概率
$$p(c_i|w)=\frac{p(w|c_i)p(c_i)}{p(w)}$$
区分正类和负类的概率即两种结果的概率

In [1]:
from numpy import *
def loadDataSet(): #词表转换成向量
    postingList = [['my','dog','has','flea','problems','help','please'],
                  ['maybe','not','take','him','to','dog','park','stupid'],
                  ['my','dalmation','is','so','cute','I','love','him'],
                  ['stop','posting','stupid','worthless','garbage'],
                  ['mr','licks','ate','my','steak','how','to','stop','him'],
                  ['quit','buying','worthless','dog','food','stupid']]
    classVec = [0,1,0,1,0,1] #1表示侮辱性文字，0代表正常言论
    return postingList,classVec

In [2]:
def createVocabList(dataSet):#取出数据集所有单词标签作为列表
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet | set(document) #“|”求集合的并集
    return list(vocabSet)

In [3]:
def setOfWords2Vec(vocabList,inputSet):#返回输入文档中单词出现次数统计矩阵
    returnVec = [0]*len(vocabList) #数据等长0向量列表作为初始化所有单词次数
    for word in inputSet: #遍历inputSet，统计vocabList词汇表中出现的次数
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else:
            print("the word: %s is not in my Vocabulary" % word)
    return returnVec

In [4]:
def trainNB0(trainMatrix,trainCategory):
    #trainCategory：统计的是否侮辱性文本，如[0,1,0,1]，1表示侮辱性文本
    numTrainDocs = len(trainMatrix) #训练文本数
    numWords = len(trainMatrix[0]) #每个文本中的单词数
    pAbusive = sum(trainCategory)/float(numTrainDocs) #侮辱性文本/总文本 ，即侮辱性文本概率
    p0Num = ones(numWords) #正常单词统计初始化
    p1Num = ones(numWords) #侮辱性单词统计初始化
    p0Denom = 2.0 
    p1Denom = 2.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:#侮辱性文本
            p1Num += trainMatrix[i] #单词向量统计
            p1Denom += sum(trainMatrix[i]) #总单词量统计作为分母
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = log(p1Num/p1Denom)#侮辱性文档中，每个侮辱性单词出现概率，用矩阵表示
    p0Vect = log(p0Num/p0Denom)
    return p0Vect,p1Vect,pAbusive

In [5]:
def classifyNB(vec2Classify,p0Vec,p1Vec,pClass1):
    #p0Vec 正常文档
    #p1Vec 侮辱性文档
    #pClass1 侮辱性文档概率
    p1 = sum(vec2Classify * p1Vec) + log(pClass1) #加log部分的前提是p1Vec就是log那么有log(a*b)=loga + logb
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else : 
        return 0

In [6]:
def testingNB():
    listOPosts,listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat = []
    for postinDoc in listOPosts:#训练统计矩阵，每组文本中单词的出现次数统计
        trainMat.append(setOfWords2Vec(myVocabList,postinDoc))
    p0V,p1V,pAb = trainNB0(array(trainMat),array(listClasses))
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = array(setOfWords2Vec(myVocabList,testEntry))
    print(testEntry,'classified as:',classifyNB(thisDoc,p0V,p1V,pAb))
    testEntry = ['stupid','garbage']
    thisDoc = array(setOfWords2Vec(myVocabList,testEntry))
    print(testEntry,'classified as:',classifyNB(thisDoc,p0V,p1V,pAb))
testingNB()

# 使用朴素贝叶斯对电子邮件进行分类

In [8]:
# 切分数据
def split_mail(mail_file):
    import re
    words_list = []
    pattern = re.compile('\W+')
    try:
        with open(mail_file,'r') as fr:
            words=re.split(pattern,fr.read())
            words_list=[word.lower() for word in words if len(word)>2]
    except UnicodeDecodeError:
        with open(mail_file,'r', encoding='Windows 1252') as fr:
            words=re.split(pattern,fr.read())
            words_list=[word.lower() for word in words if len(word)>2]
    return words_list

In [9]:
def spamTest():
    import random
    classList = [] #分类标签
    docList = [] #文档列表
    fullText = []
    for fn in range(1,26): #打开目录下所有文件获取词汇
        wordList = split_mail('email/ham/%d.txt' % (fn))
        docList.append(wordList)
        classList.append(1)
        wordList = split_mail('email/spam/%d.txt' % (fn))
        docList.append(wordList)
#         fullText.extend(wordList)
        classList.append(0)
    # 词汇表 
    vocabList = createVocabList(docList)
    testSet = [int(num) for num in random.sample(range(50), 10)]#生成50以内的随机10个数
    trainingSet = list(set(range(50)) - set(testSet)) #将训练集剔除测试集
    trainMat = []
    trainClasses = []
    for docIndex in trainingSet:
        trainMat.append(setOfWords2Vec(vocabList,docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam = trainNB0(array(trainMat),array(trainClasses))
    errorCount = 0
    for docIndex in testSet:
        wordVector = setOfWords2Vec(vocabList,docList[docIndex])
        if classifyNB(array(wordVector),p0V,p1V,pSpam) != classList[docIndex]:
            errorCount += 1
    print('the errorCount is :',errorCount)
    print('the testSet length is :',len(testSet))
    print('the error rate is :',float(errorCount)/len(testSet))
    
spamTest()  

the errorCount is : 0
the testSet length is : 10
the error rate is : 0.0


In [10]:
def data_test():
    x1 = ['1','1','1','1','1','2','2','2','2','2','2','3','3','3','3','3']
    x2 = ['S','L','M','M','S','L','S','S','L','L','M','M','L','S','M','M']
    y = ['-1','1','1','-1','-1','1','1','-1','1','-1','1','1','1','1','-1','1']
    x3 = [(x1[z],x2[z]) for z in range(len(x1))]
    return x3,y

def testing2():
    listOPosts,listClasses = data_test()
    print(listOPosts)
    print(listClasses)
    myVocabList = createVocabList(listOPosts)
    trainMat = []
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList,postinDoc))
    p0V,p1V,pAb = trainNB0(array(trainMat),array(listClasses))
    testEntry = [('2','S')] 
    thisDoc = array(setOfWords2Vec(myVocabList,testEntry))
    print(testEntry,'classified as:',classifyNB(thisDoc,p0V,p1V,pAb))
#应该要改下classifyNB公式，多计算一组概率
testing2() 

[('1', 'S'), ('1', 'L'), ('1', 'M'), ('1', 'M'), ('1', 'S'), ('2', 'L'), ('2', 'S'), ('2', 'S'), ('2', 'L'), ('2', 'L'), ('2', 'M'), ('3', 'M'), ('3', 'L'), ('3', 'S'), ('3', 'M'), ('3', 'M')]
['-1', '1', '1', '-1', '-1', '1', '1', '-1', '1', '-1', '1', '1', '1', '1', '-1', '1']


TypeError: cannot perform reduce with flexible type

In [ ]:
x1 = array(['1','1','1','1','1','2','2','2','2','2','2','3','3','3','3','3'])
x2 = array(['S','L','M','M','S','L','S','S','L','L','M','M','L','S','M','M'])
y = array(['-1','1','1','-1','-1','1','1','-1','1','-1','1','1','1','1','-1','1'])
x3=vstack((x1,x2))
x3[:,0]

In [ ]:
x1 = ['1','1','1','1','1','2','2','2','2','2','2','3','3','3','3','3']
x2 = ['S','L','M','M','S','L','S','S','L','L','M','M','L','S','M','M']
y = ['-1','1','1','-1','-1','1','1','-1','1','-1','1','1','1','1','-1','1']
x3 = [(x1[z],x2[z]) for z in range(len(x1))]
x3